In [1]:
line_data = 'C:/Users/jackp/Research/spatical_scan/file.txt'
repo_name = 'C:/Users/jackp/Research/repository/android-job'
until_data = '2019-6-18'
since_data = '2018-6-11'
dot_name = 'C:/Users/jackp/Research/spatical_scan/all_jar.dot'

In [2]:
#変更されたノードの取得
from git import *
import datetime, time


def change_node(repo_name, until_data, since_data, branch_name):

    #コミットリストの作成
    list1 = []
    repo = Repo(repo_name)
    #print('hexsha,author,authored_date,file_name,deletions,lines,insertions')
    for item in repo.iter_commits(branch_name , until = until_data ,since = since_data):
        file_list = item.stats.files
        for file_name in file_list:
        #    dt = datetime.datetime.fromtimestamp(item.authored_date).strftime("%Y-%m-%d %H:%M:%S")
        #    insertions = file_list.get(file_name).get('insertions')
        #    deletions = file_list.get(file_name).get('deletions')
        #    lines = file_list.get(file_name).get('lines')
             list1.append(file_name)

    list1 =list(set(list1))
    
    return list1

In [3]:
node_list = change_node(repo_name, until_data, since_data, branch_name = 'master')

In [4]:
node_list =list(set(node_list))

In [5]:
#テキストデータ読み込み
col0 = []
col1 = []
for i, line in enumerate(open(line_data,'r',encoding="utf-8")): # ファイルを開いて一行一行読み込む
    c = line.lstrip().split(" ")
    c[0]=c[0].strip()
    c[1]=c[1].strip()
    col0.append((c[0]))
    col1.append((c[1]))


#ファイル行数のリストを作成
file_lines = []
for x in col1:
    file_lines.append(x)        # x : ファイル名   y : ファイル行数

In [8]:
#txtファイル→グラフへの変換
col0 = [] #始点の頂点集合
col1 = [] #終点の頂点集合
for i, line in enumerate(open(dot_name)): # ファイルを開いて一行一行読み込む
    
    c = line.split("->")
    c[0]=c[0].strip()
    c[0]=c[0].strip('  ')
    c[0]=c[0].strip('\"')
    c[1]=c[1].strip()
    c[1]=c[1].strip('  ')
    c[1]=c[1].strip(";")
    c[1]=c[1].strip('\"')
    c[0] =  c[0].replace('.', '/')
    c[0] =  'src/' +  c[0] + '.java'
    c[0] =  c[0] + '.java'
    col0.append((c[0]))
    c[1] =  c[1].replace('.', '/')
    c[1] =  'src/' +  c[1] + '.java'
    c[1] =  c[1] + '.java'
    col1.append((c[1]))
    
#辺のリストを作成
java_module = []
for x, y in zip(col0, col1):
    java_module.append([x,y])
    
#java_moduleはモジュール依存関係のグラフ
java_module

[['src/android/support/v4/app/SafeJobIntentService.java.java',
  'src/android/os/Build.java.java'],
 ['src/android/support/v4/app/SafeJobIntentService.java.java',
  'src/android/os/Build$VERSION.java.java'],
 ['src/android/support/v4/app/SafeJobIntentService.java.java',
  'src/android/support/annotation/RestrictTo.java.java'],
 ['src/android/support/v4/app/SafeJobIntentService.java.java',
  'src/android/support/annotation/RestrictTo$Scope.java.java'],
 ['src/android/support/v4/app/SafeJobIntentService.java.java',
  'src/android/support/v4/app/JobIntentService.java.java'],
 ['src/android/support/v4/app/SafeJobIntentService.java.java',
  'src/android/support/v4/app/JobIntentService$CompatJobEngine.java.java'],
 ['src/android/support/v4/app/SafeJobIntentService.java.java',
  'src/android/support/v4/app/JobIntentService$GenericWorkItem.java.java'],
 ['src/android/support/v4/app/SafeJobIntentService.java.java',
  'src/android/support/v4/app/SafeJobServiceEngineImpl.java.java'],
 ['src/andro

In [10]:
#名前の書き換え
for i in file_lines:
    for j in java_module:
        if i.endswith(j[0]):
            j[0] = i
        elif i.endswith(j[1]):
            j[1] = i
            
#該当の依存関係の抽出
#(java_commitにあるノードに関係あるやつを見つける)

dependency = []
for i in node_list:
    for j in java_module:
        if i in j:
            dependency.append(j)

#重複排除
dependency2 = []
for i in dependency:
    if i not in dependency2:
        dependency2.append(dependency)

In [13]:
#ノードクラスの定義
class Node:
    def __init__(self, node_name):
        self.name = node_name

#エッジクラスの定義
class Edge:
    def __init__(self, start, end):
        self.start = start
        self.end = end
    def edge_elem(self):
        return [self.start, self.end]